# Welcome to ProgPy's Growth Example!

In this example, we'll be demonstrating how to use the Paris Law Crack Growth Equation for Prognostics.

The Paris Law or Paris Crack Growth Law is a seminal equation in the field of fracture mechanics. It models the growth of fatigue cracks, and is named after Paul C. Paris who first proposed it.

This law predicts the rate at which an existing crack will grow as a function of the range of stress intensity factor experienced by the crack. It is given by:

$$
\frac{d a}{d N}=C\left(\Delta K\right)^{m}
​$$


​where 
$
\frac{d a}{d N}
$
  is the crack growth per cycle, 
$
ΔK
$ is the stress intensity range, and 
$
C$ and 
$m$ are material constants. In the context of prognostics, understanding and predicting crack growth is vital to estimating the remaining useful life (RUL) of a component or system. This knowledge can help prevent catastrophic failures and optimize maintenance scheduling.

### __States in the `ParisLawCrackGrowth` Class__

The `ParisLawCrackGrowth` class also has one state, `c_l`, which stands for the current length of the crack. This state represents the internal state of the system being modeled, and it evolves over time based on the system inputs and the Paris Law equation.

### Importing Modules

In [ ]:
import csv
import matplotlib.pyplot as plt
import os
from progpy.models.experimental.paris_law import ParisLawCrackGrowth 

First, we'll define a model object.

In [ ]:
m = ParisLawCrackGrowth(process_noise = 0)

Next, we'll define the [Future Loading](https://nasa.github.io/progpy/prog_models_guide.html#future-loading) Function.

In [ ]:
def future_loading(t, x=None):
    #variable (piece-wise) future loading scheme 
    #inputs are ['k_min', 'k_max']
    if (t < 500):
        k_min = 12
        k_max = 24
    elif (t < 750):
        k_min = 8
        k_max = 32
    else:
        k_min = 0
        k_max = 28
    return m.InputContainer({'k_min': k_min, 'k_max': k_max})

### __Inputs in the `ParisLawCrackGrowth` Class__

The `ParisLawCrackGrowth` class takes two inputs:

1. `k_max`: This is the maximum stress intensity factor experienced by the crack.
2. `k_min`: This is the minimum stress intensity factor experienced by the crack.

The difference between `k_max` and `k_min` gives the range of the stress intensity factor, which is an essential part of the Paris Law equation.

In our future loading function, we are setting the growth limits to different values depending on the time relative to our model.

One caveat of our system is that we do not know the model's parameters for this system, thus we'll need to estimate it using data collected from the system. 

First, we'll have to import some data from the real system.

In [ ]:
times = []
inputs = []
outputs = []
csv_dir = 'growth.csv'

#Reads csv file
try:
    with open(csv_dir, newline='') as csvfile:
        data = csv.reader(csvfile, delimiter=',', quotechar='|' , quoting=csv.QUOTE_NONNUMERIC)
        for row in data:
            times.append(row[0])
            inputs.append({'k_min': row[1], 'k_max': row[2]})
            outputs.append({'c_l': row[3]})
except FileNotFoundError:
    print("No data file found")

Next, we can use our [Parameter Estimation](https://nasa.github.io/progpy/prog_models_guide.html#parameter-estimation) feature to estimate the parameters of the model.

In [ ]:
# Estimates the model parameters
keys = ['c', 'm']

print('Model configuration before')
for key in keys:
    print("-", key, m.parameters[key])
print(' Error: ', m.calc_error(times, inputs, outputs, dt=10))

m.estimate_params([(times, inputs, outputs)], keys, dt=10)

print('\nOptimized configuration')
for key in keys:
    print("-", key, m.parameters[key])
print(' Error: ', m.calc_error(times, inputs, outputs, dt=10))

Now that we have some model parameters and a future loading function, we can utilize our Simulation feature to run a simulation of our model to threshold!

### __Events in the `ParisLawCrackGrowth` Class__

In the `ParisLawCrackGrowth` class, there is one event defined: `CGF` or Crack Growth Fracture. This event signifies that the crack length has reached a certain limit, the `crack_limit`, beyond which the component or system is considered to have fractured or failed.

In [ ]:
options = {
    'save_freq': 10, # Frequency at which results are saved
    'dt': 10, # Timestep
    'horizon': 1e5, # Horizon
}

(times, inputs, _, outputs, event_states) = m.simulate_to_threshold(future_loading, **options)

Let's plot the results of the growth model. We'll plot the inputs, the event state, and the output of our model til threshold is met!

In [ ]:
inputs.plot(ylabel='Stress Intensity')
event_states.plot(ylabel= 'CGF')
outputs.plot(ylabel= {'c_l': "Crack Length"}, compact= False)
plt.show()

### __Outputs from the `ParisLawCrackGrowth` Class__

The `ParisLawCrackGrowth` class has one output, which is `c_l` (_the model's state_), the current length of the crack. This output is updated with each cycle based on the Paris Law equation.


### Conclusion

ProgPy users can utilize the ParisLawCrackGrowth Function to calculate the crack growth rate of a material. The user can input the stress intensity factor, the crack length, and the material properties to calculate the crack growth!

For more information on ProgPy, please refer to our ProgPy [Documentation](https://nasa.github.io/progpy/index.html).